In [2]:
import pandas as pd
from plant_model import PlantUnits
from utils import get_raw_data_by_time
from datetime import datetime, timedelta
import numpy as np
from utils import get_plant_characteristics
from utils import get_demand_data
from data_validations_preprocessing import capacity_checks_of_plants,cost_checks_of_plant,filling_missing_demand_withmean,missing_demand_at_timeblock_level_and_filling
from demand_model import Demand
from statistics import mean
from output_validations import output_formatting,capacity_constraint_check

In [3]:
COAL_RAMP_UP_PERCENT = 0.01
COAL_RAMP_DOWN_PERCENT = 0.015
COAL_EFFICIENCY_RATE = 0.55
DEVELOPMENT_PERIOD_START_TIME = datetime(2022, 1, 1)
DEVELOPMENT_PERIOD_END_TIME = datetime(2022, 2, 1)
MODEL_PERIOD_START_TIME = datetime(2021, 2, 1)
MODEL_PERIOD_END_TIME = datetime(2021, 3, 1)
INFINITE_CAPACITY = 8000
TIME_LEVEL = ['date','hour_of_day','time_block_of_day']
DEMAND_LEVEL = ['date','hour_of_day','avg_unit_current_load']
FILE_NAME = "RawData/upsldc_plant_unit_time_block.csv"

In [4]:
raw_data = pd.read_csv("RawData/upsldc_plant_unit_time_block.csv")
raw_data['date'] = pd.to_datetime(raw_data['data_capture_time_block_start']).dt.date

In [5]:
plant_unit_timeblocks = get_raw_data_by_time(raw_data,DEVELOPMENT_PERIOD_START_TIME, DEVELOPMENT_PERIOD_END_TIME)

In [6]:
model_data = get_raw_data_by_time(raw_data,MODEL_PERIOD_START_TIME, MODEL_PERIOD_END_TIME)

In [7]:
model_data.head(3)

,data_capture_time_block_start,plant_name,cea_plant_unit,actual_plant_unit,plant_ownership,plant_fuel_type,company_name,group_name,upsldc_unit_capacity,avg_unit_current_load,upsldc_plant_name,upsldc_unit_name,merit_aggregate_station_name,variable_cost,fixed_cost,hour_of_day,time_block_of_day,date
1327913,2021-02-01 00:00:00,ANPARA C TPS,1,1,PVT SECTOR,THERMAL,LANCO Infratech Ltd,LANCO,555.0,299.939209,LANCO,LANCO Unit 1,ANPARA C TPS,1.87,0.82,0,1,2021-02-01
1327914,2021-02-01 00:00:00,ANPARA C TPS,2,2,PVT SECTOR,THERMAL,LANCO Infratech Ltd,LANCO,555.0,299.822744,LANCO,LANCO Unit 2,ANPARA C TPS,1.87,0.82,0,1,2021-02-01
1327915,2021-02-01 00:00:00,ANPARA TPS,1,1,STATE SECTOR,THERMAL,UPRVUNL,UP State,192.0,0.000000,ANPARA-A,ANPARA THERMAL#A 210MW Unit 1,ANPARA - A,1.80,0.72,0,1,2021-02-01


In [8]:
plant_units = get_plant_characteristics(plant_unit_timeblocks)

/home/ubuntu/UP-Power-Scheduling/utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plant_data['upsldc_unit_capacity'] = 0.85*plant_data['upsldc_unit_capacity']
/home/ubuntu/UP-Power-Scheduling/utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plant_data['upsldc_unit_capacity'] = 0.85*plant_data['upsldc_unit_capacity']
/home/ubuntu/UP-Power-Scheduling/utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [9]:
demand_of_UP_bydate_byhour_units,demand_UP = get_demand_data(model_data)

In [10]:
print(capacity_checks_of_plants(plant_units))
print(cost_checks_of_plant(plant_units))

capacity checks done
This plant unit has wrong capacityPARICHHA TPS unit:2


In [11]:
average_demand_dict=filling_missing_demand_withmean(demand_of_UP_bydate_byhour_units)
demand_of_UP_bydate_byhour_units_filled,demand_values = missing_demand_at_timeblock_level_and_filling(demand_of_UP_bydate_byhour_units,average_demand_dict)


/home/ubuntu/UP-Power-Scheduling/data_validations_preprocessing.py:52: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  mapping_hour_time_block = pd.read_csv("RawData/hours_timeblock_mapping.csv",header=None,index_col=0,squeeze=True).to_dict()


In [12]:
plant_names = []
plant_production_costs = {}
plant_capacity = {}
plant_ramp_up_deltas = {}
plant_ramp_down_deltas = {}
for plant in plant_units:
    if(plant.average_variable_cost>0):
        plant_names.append(plant.name)
        plant_production_costs[plant.name] = plant.average_variable_cost
        plant_capacity[plant.name] = plant.capacity
        plant_ramp_up_deltas[plant.name] = plant.ramp_up_delta
        plant_ramp_down_deltas[plant.name] = plant.ramp_down_delta

scheduling_time_blocks = list(range(1,97))
scheduling_dates =  sorted(demand_UP['date'].unique())



In [13]:
from pulp import *
#creating the LP problem instance
prob = LpProblem("production_cost_minimization", LpMinimize)


#Creating the production variables
production_vars = LpVariable.dicts(name = "production_units",indices = [(i,j,k) for i in plant_names for j in scheduling_dates for k in scheduling_time_blocks],lowBound=0,cat = 'Continuous')
#ramping_binaries = LpVariable.dicts(name = "ramping_binaries",indices = [(i,j,k) for i in plant_names for j in scheduling_dates for k in scheduling_time_blocks],upBound=1,cat='Integer')


#LP Objective function 
prob += lpSum([plant_production_costs[i]*production_vars[(i,j,k)] for i in plant_names for j in scheduling_dates for k in scheduling_time_blocks]), "Sum of production costs"

for date in scheduling_dates:
        for time_block in scheduling_time_blocks:
            demand_date_timeblock = str(date) + "-"+str(time_block)
            prob+= (lpSum(production_vars[(plant,date,time_block)] for plant in plant_names) >= demand_values[demand_date_timeblock])


# capacity constraint 
# capacity of the power plant cannot be exceeded at any hour

for plant in plant_names:
    for date in scheduling_dates:
        for time_block in scheduling_time_blocks:
                prob += production_vars[(plant,date,time_block)] <= plant_capacity[plant]

#ramp up constraints 
for plant in plant_names:
    if plant != 'UP DRAWAL unit:0':
        for date in scheduling_dates:
            for time_block in scheduling_time_blocks[:-1]:
                prob += production_vars[(plant,date,time_block+1)] - production_vars[(plant,date,time_block)]<= plant_ramp_up_deltas[plant]

#ramp up constraints for the last time block connecting to the next day
for plant in plant_names:
    if plant != 'UP DRAWAL unit:0':
        for today in scheduling_dates[:-1]:
            tomorrow = today+timedelta(1)
            prob += production_vars[(plant,tomorrow,1)] - production_vars[(plant,today,96)]<= plant_ramp_up_deltas[plant]

#ramp down constraints 
for plant in plant_names:
    if plant != 'UP DRAWAL unit:0':
        for date in scheduling_dates:
            for time_block in scheduling_time_blocks[:-1]:
                prob +=  production_vars[(plant,date,time_block)] - production_vars[(plant,date,time_block+1)]<= plant_ramp_down_deltas[plant]

#ramp up constraints for the last time block connecting to the next day
for plant in plant_names:
    if plant != 'UP DRAWAL unit:0':
        for today in scheduling_dates[:-1]:
            tomorrow = today+timedelta(1)
            prob += production_vars[(plant,today,96)] - production_vars[(plant,tomorrow,1)]<= plant_ramp_down_deltas[plant]

In [14]:
prob.writeLP('prob.lp')

[production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_1),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_10),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_11),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_12),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_13),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_14),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_15),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_16),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_17),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_18),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_19),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_2),
 production_units_('ANPARA_C_TPS_unit:1',_datetime.date(2021,_2,_1),_20),
 production_units_('ANPARA_C_TPS_unit:1'

In [15]:
#solving 
prob.solve()
    
#status of solving the optimization problem
print("Status:", LpStatus[prob.status])

#optimal objective value
print("Total cost of production = ", value(prob.objective))



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ubuntu/anaconda3/envs/uppower/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/2e181b5464c2453fabff771926ca95ba-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/2e181b5464c2453fabff771926ca95ba-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 545535 COLUMNS
At line 1814004 RHS
At line 2359535 BOUNDS
At line 2359536 ENDATA
Problem MODEL has 545530 rows, 182784 columns and 1085684 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 362738 (-182792) rows, 182776 (-8) columns and 902356 (-183328) elements
Perturbing problem by 0.001% of 4.4571588 - largest nonzero change 0.00074496384 ( 0.13238545%) - largest zero change 0
0  Obj 4286863.2 Primal inf 25681564 (2680)
1000  Obj 15027281 Primal inf 22385240 (10730)
2000  Obj 23179311 Primal inf 18903371 (12186)
3000  Obj 3005